# Exploring the Data

## 1.1 Load the Datasets
- Load Box Office Mojo and IMDB datasets into pandas dataframes using pd.read_csv() and pd.read_sql ()
- Use head(), info(), and describe() methods to get a quick overview of each dataset.

In [1]:
#Import and load the data with pandas and sqlite3
import sqlite3
import pandas as pd

### Box office Data

In [2]:
box_office_data = pd.read_csv('../data/bom.movie_gross.csv.gz')

In [3]:
box_office_data.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [4]:
box_office_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [5]:
box_office_data.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


### Imdb Data

In [6]:
# Connect to the database
conn = sqlite3.connect('../data/im.db')

In [7]:
#View all data from sqlite_master such as table names
query = "SELECT * FROM sqlite_master"


In [8]:
# Load the data into a pandas DataFrame
imdb_data = pd.read_sql(query, conn)
imdb_data

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


## 1.2 Understand the structure:
- Identify the key features in each dataset (e.g., movie title, genre, budget, revenue, etc.).
- Use value_counts() to look at the distribution of key categorical variables like genre or director.

### Box Office Data Key Features

- title (movie title)
- studio (movie production studio)
- domestic_gross (revenue from domestic box office)
- foreign_gross (revenue from foreign box office)
- year (release year)

Key Data Insights:

- Some missing values in studio and domestic_gross.
- Significant missing values in foreign_gross.
- Data types are generally correct except for foreign_gross, which is stored as object but should likely be float64 to handle numeric operations.

### IMDB Features
- movie_basics: Contains key information about movies like movie_id, title, genre, runtime_minutes, start_year, etc.
- directors: Links directors to movies via movie_id.
- known_for: Associates people (person_id) with movies.
- movie_ratings: Contains information about movie ratings (average_rating, num_votes).
- persons: Holds person-specific details such as name, birth_year, death_year, etc.
- principals: Contains cast and crew information for each movie.
- writers: Links writers to movies via movie_id.


### Data Distribution of Key Categorical Variables (Box Office Data)

In [9]:
# Distribution of studios
print(box_office_data['studio'].value_counts())

# Distribution of years
print(box_office_data['year'].value_counts())


studio
IFC           166
Uni.          147
WB            140
Fox           136
Magn.         136
             ... 
E1              1
PI              1
ELS             1
PalT            1
Synergetic      1
Name: count, Length: 257, dtype: int64
year
2015    450
2016    436
2012    400
2011    399
2014    395
2013    350
2010    328
2017    321
2018    308
Name: count, dtype: int64


This would show which studios and years are most represented in the dataset, which can help analyze trends over time or by studio.

### Distribution of Key Categorical Variables (IMDB Database):


In [10]:
# Distribution of genres in movie_basics
query1 = "SELECT genres FROM movie_basics"
movie_genres = pd.read_sql(query1, conn)
print(movie_genres['genres'].value_counts())

# Distribution of directors
query2 = "SELECT primary_name FROM persons JOIN directors USING(person_id)"
director_names = pd.read_sql(query2, conn)
print(director_names['primary_name'].value_counts())


genres
Documentary                   32185
Drama                         21486
Comedy                         9177
Horror                         4372
Comedy,Drama                   3519
                              ...  
Adventure,Music,Mystery           1
Documentary,Horror,Romance        1
Sport,Thriller                    1
Comedy,Sport,Western              1
Adventure,History,War             1
Name: count, Length: 1085, dtype: int64
primary_name
Tony Newton          238
Jason Impey          190
Shane Ryan           186
Ruben Rodriguez      181
Sam Mason-Bell       144
                    ... 
Muta'Ali Muhammad      1
Maureen Maundu         1
Michael A. Ybarra      1
Bulmaro Osornio        1
Luis E. Froiz          1
Name: count, Length: 106757, dtype: int64


This gives an idea of which genres are most common and which directors have worked on the most films.

## 1.3 Identify Relationships Between Datasets
Merging Box Office Mojo with IMDB Database:
 - Possible keys: Merge the Box Office Mojo dataset and the IMDB data using the title field from Box Office Mojo and the title field in the movie_basics table. Another option could be to use the movie_id from the IMDB database if it exists in both datasets.

Merging Tables within the IMDB Database:

- The tables within the IMDB database can be merged using movie_id to combine relevant information

  - movie_basics with movie_ratings to get both the movie details and their ratings.
  - movie_basics with directors, writers, or principals to understand crew members associated with each movie.

## 1.4 Outline Business Questions:
- What genres perform best at the box office?
   - Analyze which genres generate the highest domestic and foreign box office revenue. 
   - Use the genre field from the movie_basics table and the domestic_gross, foreign_gross from the box office data.
- What factors influence a movie’s success (budget, genre, director, etc.)?
   - Investigate how different factors such as budget (from the IMDB database), genre, and director influence a movie's success in terms of revenue or ratings.
- Which movies have the highest ROI (return on investment)?
   - Calculate ROI for movies using the formula:
   - ROI= (domestic_gross+foreign_gross−budget) / budget
   - Analyze which genres or studios tend to produce the highest ROI, providing a cost-benefit perspective to movie production.

# Data Cleaning

## 2.1 Handle Incorrect Data Types
- Convert columns to their correct data types. 
- Convert the foreign_gross column to numeric values since it's currently stored as an object.

In [11]:
# Convert 'foreign_gross' to numeric, coerce errors
box_office_data['foreign_gross'] = pd.to_numeric(box_office_data['foreign_gross'], errors='coerce')
(box_office_data['foreign_gross'])

0       652000000.0
1       691300000.0
2       664300000.0
3       535700000.0
4       513900000.0
           ...     
3382            NaN
3383            NaN
3384            NaN
3385            NaN
3386            NaN
Name: foreign_gross, Length: 3387, dtype: float64

## 2.2 Handle Missing Values
As mentioned earlier, there are missing values in the studio, domestic_gross and in foreign_gross columns in the box office data.

- Use isnull() and sum() to identify columns with missing data.

Depending on the context:
- Drop rows or columns with a large amount of missing data using dropna().
- Impute missing values with appropriate statistics (mean, median, mode) using fillna().


1. studio (5 missing values):
- Fill in the missing values with "Unknown" rather than dropping them. These movies still have important data like gross earnings, which is crucial for analysis.
  - The studio is not the primary focus of our analysis, and removing these rows could unnecessarily reduce the size of our dataset.

In [12]:
box_office_data['studio'].fillna('Unknown', inplace=True)
box_office_data['studio']


/var/folders/p9/l56kxrqj1f50k63kvkm8k0nm0000gp/T/ipykernel_16512/1725922360.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  box_office_data['studio'].fillna('Unknown', inplace=True)


0               BV
1               BV
2               WB
3               WB
4             P/DW
           ...    
3382         Magn.
3383            FM
3384          Sony
3385    Synergetic
3386         Grav.
Name: studio, Length: 3387, dtype: object

2. domestic_gross (28 missing values):
- Drop rows where domestic_gross is missing.
  - Missing domestic gross values make it impossible to assess a movie’s financial performance, which is essential for our analysis. Imputing a value here (e.g., with a mean or median) could distort our analysis.

In [13]:
box_office_data = box_office_data.dropna(subset=['domestic_gross'])
box_office_data['domestic_gross']


0       415000000.0
1       334200000.0
2       296000000.0
3       292600000.0
4       238700000.0
           ...     
3382         6200.0
3383         4800.0
3384         2500.0
3385         2400.0
3386         1700.0
Name: domestic_gross, Length: 3359, dtype: float64

3. foreign_gross (1350 missing values):
- Fill in missing foreign_gross values with 0.
  - While a missing foreign gross could imply that the movie was not released internationally, setting the value to 0 allows us to continue analyzing its total performance, especially if it performed well domestically.

In [14]:
# check to see if there are any null values in foreign_gross
box_office_data['foreign_gross'].isna().sum()

1355

In [15]:
# replace these null values with 0
box_office_data["foreign_gross"].fillna(0, inplace=True)

box_office_data['foreign_gross'].isna().sum()

/var/folders/p9/l56kxrqj1f50k63kvkm8k0nm0000gp/T/ipykernel_16512/3693978893.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  box_office_data["foreign_gross"].fillna(0, inplace=True)


0

## 2.3 Splitting and Normalizing the Genres Columns
- Problem: Movies might be listed with multiple genres (e.g., Action, Comedy), so we need to normalize the genre data for easier analysis.
- Solution: Split the genres into separate rows so that each movie has one genre per row.

In [16]:
# putting all movie_basics info inside 1 df
movie_basics_df = pd.read_sql(
"""
SELECT *
FROM movie_basics
"""
, conn)
movie_basics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [17]:
# Split the 'genres' column by commas
movie_basics_df['individual_genre'] = movie_basics_df['genres'].str.split(',')

# Explode the list of genres into individual rows
movie_basics_df = movie_basics_df.explode('individual_genre')
movie_basics_df

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,individual_genre
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",Action
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",Crime
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",Drama
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",Biography
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",Drama
...,...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None,None


In [18]:
# check to see if there are null values in individual_genre
movie_basics_df["individual_genre"].isna().sum()

5408

In [19]:
# replace null values in individual_genre with "Unknown"
movie_basics_df["individual_genre"].fillna("Unknown", inplace=True)

movie_basics_df["individual_genre"].isna().sum()

/var/folders/p9/l56kxrqj1f50k63kvkm8k0nm0000gp/T/ipykernel_16512/1637370454.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movie_basics_df["individual_genre"].fillna("Unknown", inplace=True)


0

This step ensures that genre analysis will be more accurate, as movies with multiple genres will be counted individually for each genre.

## 2.4 Checking for Duplicate or Irrelevant Data:
- Ensure there are no duplicate rows in relevant databases
- Remove irrelevant columns that are not needed for analysis 


## 2.5 Handling Director Information
- Instead of relying on person_id, we can join the directors table with the persons table to get director names for easier interpretation.
- This will be helpful when identifying relationships between directors and movie success.

In [20]:
# get director name of each movie
query3 = """
SELECT DISTINCT movie_id, person_id AS director_id, primary_name AS director_name
FROM persons JOIN directors USING(person_id)
"""
movie_director_details = pd.read_sql(query3, conn)
movie_director_details


,movie_id,director_id,director_name
0,tt1592569,nm0062879,Ruel S. Bayani
1,tt2057445,nm0062879,Ruel S. Bayani
2,tt2590280,nm0062879,Ruel S. Bayani
3,tt8421806,nm0062879,Ruel S. Bayani
4,tt3501180,nm0064023,Bryan Beasley
...,...,...,...
163528,tt8697720,nm9971456,Zheng Wei
163529,tt8715016,nm9980896,Rama Narayanan
163530,tt8919136,nm9980896,Rama Narayanan
163531,tt8717234,nm9981679,Samir Eshra


# Data Transformation
Filter the datasets to include only the most relevant features for our analysis.

## 3.1 Filter for relevant features:
From Box Office Mojo Data:
- title: This is necessary to match movies with other datasets.
- studio: analyze which studios are producing successful movies.
- domestic_gross: This is the revenue generated domestically, which is a key measure of box office performance.
- foreign_gross: Revenue generated in foreign markets, another important aspect of box office success.
- year: Useful to analyze trends over time.

In [21]:
# Select relevant columns from Box Office Mojo data
box_office_filtered = box_office_data[['title', 'studio', 'domestic_gross', 'foreign_gross', 'year']]
box_office_filtered


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,Inception,WB,292600000.0,535700000.0,2010
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,0.0,2018
3383,Edward II (2018 re-release),FM,4800.0,0.0,2018
3384,El Pacto,Sony,2500.0,0.0,2018
3385,The Swan,Synergetic,2400.0,0.0,2018


From IMDB SQL Database:
- movie_basics (from movie_basics table):
  - genres: Key feature to analyze which genres are most successful.
  - runtime_minutes: This could be useful for analyzing whether longer or shorter movies perform better.
  - title: To match with the Box Office data.

- movie_ratings (from movie_ratings table):
  - average_rating: This is the IMDB rating, useful to analyze the relationship between ratings and success.
  - num_votes: Number of votes can help measure how popular or widely seen the movie is.
- directors (optional for analyzing the influence of directors):
  - person_id: This can be linked to the persons table to get the director's name and further explore the influence of certain directors.

In [22]:
# Filter data from the cleaned df -- movie_basics_df 

movie_basics_filtered = movie_basics_df[["movie_id", "primary_title", "genres", "individual_genre", "runtime_minutes"]]
movie_basics_filtered

,movie_id,primary_title,genres,individual_genre,runtime_minutes
0,tt0063540,Sunghursh,"Action,Crime,Drama",Action,175.0
0,tt0063540,Sunghursh,"Action,Crime,Drama",Crime,175.0
0,tt0063540,Sunghursh,"Action,Crime,Drama",Drama,175.0
1,tt0066787,One Day Before the Rainy Season,"Biography,Drama",Biography,114.0
1,tt0066787,One Day Before the Rainy Season,"Biography,Drama",Drama,114.0
...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Drama,Drama,123.0
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Documentary,Documentary,NaN
146141,tt9916706,Dankyavar Danka,Comedy,Comedy,NaN
146142,tt9916730,6 Gunn,None,Unknown,116.0


In [23]:
query_movie_ratings = """
SELECT movie_id, averagerating, numvotes
FROM movie_ratings
"""
movie_ratings_filtered = pd.read_sql(query_movie_ratings, conn)

movie_ratings_filtered


,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


Join the Box Office and IMDB Data:

Join on title (or movie_id if possible) to merge the Box Office data with the relevant IMDB data.

In [24]:
movie_basics_filtered.head()

,movie_id,primary_title,genres,individual_genre,runtime_minutes
0,tt0063540,Sunghursh,"Action,Crime,Drama",Action,175.0
0,tt0063540,Sunghursh,"Action,Crime,Drama",Crime,175.0
0,tt0063540,Sunghursh,"Action,Crime,Drama",Drama,175.0
1,tt0066787,One Day Before the Rainy Season,"Biography,Drama",Biography,114.0
1,tt0066787,One Day Before the Rainy Season,"Biography,Drama",Drama,114.0


In [25]:
box_office_filtered.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000.0,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010
3,Inception,WB,292600000.0,535700000.0,2010
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010


In [26]:
# Merge Box Office and IMDB data
merged_data = movie_basics_filtered
merged_data = merged_data.merge(box_office_filtered, left_on="primary_title", right_on="title", suffixes=("_movie", "_bo"))

# Adding rating data to merged_data df
merged_data = merged_data.merge(movie_ratings_filtered, on="movie_id", suffixes=("_movie", "_rating"))

# adding director details to merged_data
merged_data = merged_data.merge(movie_director_details, on="movie_id", suffixes=("_movie", "_dir"))

# there are 2999 unique movies in merged_data
print(len(merged_data["movie_id"].unique()))

merged_data

2999


,movie_id,primary_title,genres,individual_genre,runtime_minutes,title,studio,domestic_gross,foreign_gross,year,averagerating,numvotes,director_id,director_name
0,tt0315642,Wazir,"Action,Crime,Drama",Action,103.0,Wazir,Relbig.,1100000.0,0.0,2016,7.1,15378,nm2349060,Bejoy Nambiar
1,tt0315642,Wazir,"Action,Crime,Drama",Crime,103.0,Wazir,Relbig.,1100000.0,0.0,2016,7.1,15378,nm2349060,Bejoy Nambiar
2,tt0315642,Wazir,"Action,Crime,Drama",Drama,103.0,Wazir,Relbig.,1100000.0,0.0,2016,7.1,15378,nm2349060,Bejoy Nambiar
3,tt0337692,On the Road,"Adventure,Drama,Romance",Adventure,124.0,On the Road,IFC,744000.0,8000000.0,2012,6.1,37886,nm0758574,Walter Salles
4,tt0337692,On the Road,"Adventure,Drama,Romance",Drama,124.0,On the Road,IFC,744000.0,8000000.0,2012,6.1,37886,nm0758574,Walter Salles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7764,tt9447594,The Gambler,"Action,Sci-Fi,Thriller",Thriller,121.0,The Gambler,Par.,33700000.0,5600000.0,2014,6.1,10,nm6981787,Tom Emmatty
7765,tt9816988,Gold,Drama,Drama,NaN,Gold,Wein.,7200000.0,7700000.0,2017,5.2,26,nm0787526,Parviz Shahbazi
7766,tt9851050,Sisters,"Action,Drama",Action,NaN,Sisters,Uni.,87000000.0,18000000.0,2015,4.7,14,nm1272773,Prachya Pinkaew
7767,tt9851050,Sisters,"Action,Drama",Drama,NaN,Sisters,Uni.,87000000.0,18000000.0,2015,4.7,14,nm1272773,Prachya Pinkaew


## 3.2 Create New Features:
Return on Investment (ROI): A critical metric to see how profitable a movie is.

In [27]:
# creating new column total_gross for domestic + foreign revenue
merged_data["total_gross"] = merged_data["domestic_gross"] + merged_data["foreign_gross"]
merged_data

,movie_id,primary_title,genres,individual_genre,runtime_minutes,title,studio,domestic_gross,foreign_gross,year,averagerating,numvotes,director_id,director_name,total_gross
0,tt0315642,Wazir,"Action,Crime,Drama",Action,103.0,Wazir,Relbig.,1100000.0,0.0,2016,7.1,15378,nm2349060,Bejoy Nambiar,1100000.0
1,tt0315642,Wazir,"Action,Crime,Drama",Crime,103.0,Wazir,Relbig.,1100000.0,0.0,2016,7.1,15378,nm2349060,Bejoy Nambiar,1100000.0
2,tt0315642,Wazir,"Action,Crime,Drama",Drama,103.0,Wazir,Relbig.,1100000.0,0.0,2016,7.1,15378,nm2349060,Bejoy Nambiar,1100000.0
3,tt0337692,On the Road,"Adventure,Drama,Romance",Adventure,124.0,On the Road,IFC,744000.0,8000000.0,2012,6.1,37886,nm0758574,Walter Salles,8744000.0
4,tt0337692,On the Road,"Adventure,Drama,Romance",Drama,124.0,On the Road,IFC,744000.0,8000000.0,2012,6.1,37886,nm0758574,Walter Salles,8744000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7764,tt9447594,The Gambler,"Action,Sci-Fi,Thriller",Thriller,121.0,The Gambler,Par.,33700000.0,5600000.0,2014,6.1,10,nm6981787,Tom Emmatty,39300000.0
7765,tt9816988,Gold,Drama,Drama,NaN,Gold,Wein.,7200000.0,7700000.0,2017,5.2,26,nm0787526,Parviz Shahbazi,14900000.0
7766,tt9851050,Sisters,"Action,Drama",Action,NaN,Sisters,Uni.,87000000.0,18000000.0,2015,4.7,14,nm1272773,Prachya Pinkaew,105000000.0
7767,tt9851050,Sisters,"Action,Drama",Drama,NaN,Sisters,Uni.,87000000.0,18000000.0,2015,4.7,14,nm1272773,Prachya Pinkaew,105000000.0


In [28]:
# check to see if there are null values
merged_data["total_gross"].isnull().sum()

0

## 3.3 Transform Categorical Variables
One-Hot Encoding for Genres: Convert genres into a numeric format for analysis, as genre is often a categorical variable that may need to be converted for modeling.

In [29]:
merged_data["individual_genre"].value_counts()

individual_genre
Drama          1894
Comedy         1081
Action          689
Adventure       517
Romance         489
Thriller        482
Crime           397
Documentary     317
Biography       305
Horror          274
Animation       227
Mystery         223
Fantasy         188
Sci-Fi          145
History         142
Family          137
Music            95
Sport            60
War              53
Western          22
Musical          19
Unknown           7
News              6
Name: count, dtype: int64

In [30]:
# Saving merged_data dataframe to use in data_analysis_notebook.ipynb
merged_data.to_pickle("merged_data.pkl")